# DATA cleaning

In [11]:
import pandas as pd
import os
import numpy as np
from copy import deepcopy
from sklearn.preprocessing import StandardScaler


## Loading the three CSV files: train.csv, cddd.csv and test.csv

In [12]:
datas_folder = "raw_datas/"
train_data_file = "train.csv"
cddd_feature_file = "cddd.csv" 
test_data_file = "test.csv"

train_data = pd.read_csv(datas_folder + train_data_file)
cddd_data = pd.read_csv(datas_folder + cddd_feature_file)
test_data = pd.read_csv(datas_folder + test_data_file)

cddd_data.describe()

,cddd_1,cddd_2,cddd_3,cddd_4,cddd_5,cddd_6,cddd_7,cddd_8,cddd_9,cddd_10,...,cddd_503,cddd_504,cddd_505,cddd_506,cddd_507,cddd_508,cddd_509,cddd_510,cddd_511,cddd_512
count,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,...,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000
mean,-0.432918,0.036407,0.302930,-0.076635,0.100060,-0.143191,0.045221,-0.098932,0.112883,-0.094099,...,0.082483,-0.059233,0.049412,-0.012243,-0.840885,-0.190269,0.255021,0.175835,0.424142,0.162907
std,0.398553,0.338912,0.327502,0.336897,0.365662,0.347969,0.352105,0.357539,0.464354,0.337649,...,0.382256,0.360114,0.348578,0.346110,0.364443,0.340333,0.333545,0.324856,0.290428,0.358960
min,-0.939515,-0.875421,-0.747219,-0.849436,-0.842215,-0.898292,-0.869110,-0.836164,-0.881907,-0.870063,...,-0.909627,-0.862160,-0.856511,-0.849983,-0.999870,-0.859648,-0.788223,-0.774710,-0.838194,-0.843842
25%,-0.773065,-0.194372,0.096958,-0.327889,-0.155688,-0.410974,-0.219453,-0.367519,-0.276358,-0.344816,...,-0.181692,-0.311846,-0.191050,-0.279613,-0.997693,-0.473120,0.040941,-0.056778,0.263420,-0.099320
50%,-0.574171,0.056478,0.335329,-0.098616,0.099087,-0.164914,0.040871,-0.119129,0.089327,-0.107351,...,0.064524,-0.079485,0.061064,-0.032866,-0.994138,-0.206405,0.303599,0.196021,0.502088,0.184956
75%,-0.149069,0.277491,0.555477,0.167016,0.362394,0.096065,0.322235,0.148721,0.548203,0.138810,...,0.373925,0.203732,0.303438,0.258854,-0.929408,0.064953,0.522219,0.427354,0.628405,0.456521
max,0.972314,0.801946,0.881048,0.861599,0.966514,0.921197,0.784904,0.877550,0.953488,0.900396,...,0.933929,0.839612,0.827979,0.810820,0.948019,0.789872,0.860602,0.869080,0.912181,0.942221


## ECFP data cleaning:
Checking for the presence of missing values in the training data and in the test data, but there is no missing value. The ECFP data are clean.

In [13]:
#NO MISSING DATA FOR TRAINING
missing_values = train_data.isnull().sum()
print((missing_values ==0).all()) # print true if not missing value

#NO MISSING DATA FOR TRAINING
missing_values = test_data.isnull().sum()
print((missing_values ==0).all()) # print true if not missing value

True
True


## CDDD data preparation:

### CDDD data cleaning and data preparation
Match the SMILES in the cddd.csv file with the SMILES in the training set.
Note: For CDDD test data, we fill in the missing data with 0, and for CDDD training data, we delete the missing data.

In [15]:
cddd_data = cddd_data.dropna() #remove de raw of the 14 MISSING VALUES

cddd_smiles = cddd_data["SMILES"]
cddd_values = cddd_data.drop(columns=["SMILES"])

#STANDARIZE (Z-STD) CDDD DATAS
scaler = StandardScaler() #standardization
cddd = pd.concat([cddd_smiles, cddd_values], axis=1)
index_train = pd.DataFrame(train_data.iloc[:,:5])
index_test = pd.DataFrame(test_data.iloc[:,:4])

# creation of CDDD train data and test data usable
cddd_train = pd.merge(index_train, cddd, how='left', on='SMILES')
cddd_test = pd.merge(index_test, cddd, how='left', on='SMILES')
null_rows_test = cddd_test[cddd_test.isnull().any(axis=1)].index
print("Il y a :",len(null_rows_test), "SMILES manquantes pour le test")

null_rows = cddd_train[cddd_train.isnull().any(axis=1)]
print("Il y a :",len(null_rows), "SMILES manquantes pour le train")

#Removing missing data on train set / Fill 0 for missing data on test set
cddd_test.fillna(0,inplace=True)
cddd_train.dropna(inplace=True)

missing_test_cddd = cddd_test[cddd_test.eq(0).any(axis=1)].index
print(missing_test_cddd)

Il y a : 6 SMILES manquantes pour le test
Il y a : 37 SMILES manquantes pour le train
Int64Index([279, 462, 695, 883, 913, 1202], dtype='int64')


## Data pre-processing
The one_hot_encoding_lab creates binary-coded columns for the various laboratory categories and replaces the contents of the 'lab' column with its associated binary code (for the data given in input) using one_hot encoding panda method. This function is used on the CDDD test set, the CDDD train set, the ECFP test set and the ECFP train set.

The function creat_file_csv takes a DataFrame, a file name and a folder name as input and saves the DataFrame in .csv format in the specified folder. It is used to save the pre-processing data into csv files.

In [16]:
#FEATURE ENGINEER ON DATAS
def one_hot_encoding_lab(datas) :
    datas = datas.copy()
    lab_encoded = pd.get_dummies(datas['Lab']).astype(int) #column in binary encoding for the different lab
    encoded = pd.concat([datas, lab_encoded], axis=1)
    encoded = encoded.drop('Lab', axis=1)  
    #DROP 1 VALUE TO AVOID REDONDANCE
    encoded.drop(["Trondheim"], inplace=True, axis = 1)
    return encoded

# Changing lab column values in data to binary code using the one_hot_encoding_lab function
cddd_train = one_hot_encoding_lab(cddd_train)
cddd_test = one_hot_encoding_lab(cddd_test)
cddd_train_log = deepcopy(cddd_train)

train = one_hot_encoding_lab(train_data)
test = one_hot_encoding_lab(test_data)

#LOG OF OUTPUT FOLLOWING NORMAL DISTRIBUTION
cddd_train_log["RT"] = np.log(cddd_train_log["RT"])
train_log = np.log(train_data["RT"])

# Data saving function
def creat_file_csv(data, file_name, folder_name):
    file = "{}/{}.csv".format(folder_name,file_name) 
    data.to_csv(file, index=False)

# SAVE DATAS
folder =  "clean_datas"
creat_file_csv(cddd_train_log, "cddd_train_log",folder)
creat_file_csv(cddd_test, "cddd_test", folder)
creat_file_csv(cddd_train, "cddd_train", folder )
creat_file_csv(train, "ecfp_train",folder)
creat_file_csv(test, "ecfp_test", folder)
creat_file_csv(train_log, "ecfp_train_log", folder)

cddd_train.describe()

,RT,cddd_1,cddd_2,cddd_3,cddd_4,cddd_5,cddd_6,cddd_7,cddd_8,cddd_9,...,LADR,Labor Krone,Mainz,Odense,San Francisco OCME,The University of Queensland,University Hospital of Northern Norway,University of Athens,Victorian Institute of Forensic Medicine,Zurich Institute of Forensic Medicine
count,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,...,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000,3463.000000
mean,7.391977,-0.404968,0.056549,0.307892,-0.092653,0.075707,-0.152696,0.054048,-0.121204,0.118433,...,0.016460,0.014150,0.125036,0.011262,0.123015,0.011262,0.012128,0.057176,0.050823,0.011839
std,3.771611,0.391551,0.329624,0.324357,0.333670,0.358853,0.342734,0.344828,0.351031,0.457803,...,0.127254,0.118125,0.330808,0.105538,0.328502,0.105538,0.109474,0.232212,0.219668,0.108179
min,0.515000,-0.939515,-0.858964,-0.747219,-0.835825,-0.842215,-0.898292,-0.869110,-0.836164,-0.881907,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.720000,-0.762150,-0.166393,0.095512,-0.344679,-0.171720,-0.410967,-0.203393,-0.394110,-0.271628,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.500000,-0.500531,0.067024,0.343490,-0.118073,0.062217,-0.167001,0.041928,-0.148405,0.076513,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.395000,-0.124461,0.293727,0.569445,0.146498,0.335204,0.070298,0.319690,0.143307,0.550474,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,21.800000,0.972314,0.801946,0.881048,0.861599,0.966514,0.921197,0.784904,0.877550,0.928988,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Creation of a mix dataset by merging ECFP and CDDD data
Creation of a new dataset by merging ECFP and CDDD data. The resulting data is cleaned before being saved in .csv files.

In [17]:
# MIXED DATAS
mix_train = pd.merge(train, cddd_data, how='left', on='SMILES')
mix_test = pd.merge(test, cddd_data, how='left', on='SMILES')

#Remove the null value
null_rows_test = mix_test[mix_test.isnull().any(axis=1)].index
null_rows_train = mix_train[mix_train.isnull().any(axis=1)].index

#Removing missing data on train set / Fill 0 for missing data on test set
mix_test.fillna(0,inplace=True)
mix_train.dropna(inplace=True)

# Save mixed data sets in specific files
creat_file_csv(mix_test, "mix_test", folder)
creat_file_csv(mix_train, "mix_train", folder)